In [1]:
import sys
import os
sys.path.append(os.path.join('D:\\', 'Documents', 'Programs', 'DisperScripts'))

In [2]:
from experimentor.models.devices.cameras.basler.basler import BaslerCamera
import zmq
from multiprocessing import Event
from threading import Thread
import time
import numpy as np
from calibration.models.experiment import CalibrationSetup
from experimentor import Q_

from pypylon import pylon, _genicam

In [ ]:
stop_event = Event()

In [ ]:
cam = BaslerCamera('ac')

In [ ]:
cam.initialize()

In [ ]:
cam.exposure = Q_('50ms')

In [ ]:
def print_number_frames(stop_event, url):
    topic = 'new_image'
    frame_no = 0
    context = zmq.Context()
    socket = context.socket(zmq.SUB)
    socket.connect(url)
    socket.setsockopt(zmq.SUBSCRIBE, topic.encode('utf-8'))
    img_old = None
    while not stop_event.is_set():
        event = socket.poll(0)
        if not event:
            time.sleep(.005)
            continue
        topic = socket.recv_string()
        metadata = socket.recv_json(flags=0)
        msg = socket.recv(flags=0, copy=True, track=False)
        buf = memoryview(msg)
        if img_old is None:
            img_old = np.zeros(metadata['shape'])
        img = np.frombuffer(buf, dtype=metadata['dtype'])
        img = img.reshape(metadata['shape']).copy()
        if np.all(img == img_old):
            break
        imd_old = img.copy()
        frame_no+=1
        if frame_no%100 == 0:
            print(f'Acquired {frame_no} frames')
    print(f'Acquired {frame_no} before getting repeated frames')

In [ ]:
url = cam.get_publisher_url()
port = cam.get_publisher_port()

In [ ]:
print(f'{url}:{port}')

In [ ]:
thread = Thread(target=print_number_frames, args=(stop_event, f'{url}:{port}'))
thread.start()

In [ ]:
cam.start_free_run()

In [ ]:
cam.continuous_reads()

In [ ]:
cam.stop_continuous_reads()

In [ ]:
stop_event.set()

In [ ]:
stop_event.clear()

In [ ]:
cam.finalize()

In [3]:
import yaml

In [4]:
experiment = CalibrationSetup()

In [5]:
experiment.load_configuration('../dispertech.yml', yaml.UnsafeLoader)

In [6]:
future = experiment.initialize()
while future.running():
    continue

binning from Camera Basler daA1280-54um (22487161) not Implemented
serial_number from Camera Basler daA1280-54um (22487161) not Implemented
binning from Camera Basler daA1280-54um (22487161) not Implemented
binning from Camera Basler acA1920-155um (22161031) not Implemented
serial_number from Camera Basler acA1920-155um (22161031) not Implemented


In [7]:
experiment.get_latest_image('microscope_camera')

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [8]:
experiment.start_saving_images()

tcp://localhost:54037


In [9]:
experiment.stop_saving_images()

Saving process still alive


In [10]:
experiment.finalize()